# 110. Content Based Filtering - manual feature engineering

- 사용자, 영화 및 feature 목록을 만듭니다. 사용자와 영화는 데이터베이스에서 가져왔다고 가정  

- 영화의 feature 는 수작업으로 작성되었으며 도메인 지식에 의존하여 최상의 임베딩 공간을 제공. 여기에서는 액션, 공상 과학, 코미디, 만화 및 드라마 카테고리를 사용하여 영화(및 사용자)를 설명합니다.

- 이 예에서는 데이터베이스가 아래에 나열된 4 명의 사용자와 6 개의 영화로 구성되어 있다고 가정합니다.

## Content-Based Recommender System algorithm

Step 1 : 사용자의 영화 평가를 근거로 weighted genre matrix 작성 (Genre 별로 평점 합산 및 normalize)  
Step 2 : Weighted genre matrix 로부터 User Profile 작성  
Step 3 : 사용자가 보지 않은 Movie matrix 에 User Feature Profile 을 적용하여 추천 영화 list 작성

In [1]:
# 필요한 library import
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# 사용자 4 명
users = ['길동', '영희',  '철수', '명수']  
# 영화 6 개
movies = [
    '스타워즈', '다크나이트', '쉬렉', '서울의 봄', '명량', '범죄도시'
         ]
# 영화 feature - 5 가지 장르
features = ['액션', '공상과학', '코미디', '만화영화', '드라마']
# num_movies = len(movies)

## 사용자, 영화 등급 (movie ratings) 및 특성(feature) 초기화

- 사용자의 영화 등급과 k-hot-encoding 된 영화 특성 매트릭스를 입력해야합니다.   
- users_movies matrix의 각 행은 각 영화에 대한 단일 사용자 등급 (1 ~ 10)을 나타냅니다. 0은 사용자가 해당 영화를 보거나 평가하지 않았음을 나타냅니다.   
- movies_feats matrix에는 주어진 영화 각각에 대한 feature가 포함되어 있습니다. 각 행은 6 개의 영화 중 하나를 나타내고 열은 5 개의 카테고리를 나타냅니다. 1 은 영화가 특정 장르/카테고리에 해당함을 나타냅니다.

In [3]:
# 각 행은 다른 영화에 대한 사용자의 평가를 나타냅니다.
# row - 사용자, column - 영화
users_seen_movies = np.array([
                [4,  6,  8,  0, 0, 0],
                [0,  0, 10,  0, 8, 3],
                [0,  6,  0,  0, 3, 7],
                [10, 9,  0,  5, 0, 2]], dtype=np.float32)

# 영화 전문가가 직접 hand-engineering한 k-hot encoding된 영화 특성
# row - 영화, column - ['액션', '공상과학', '코미디', '만화영화', '드라마']
movies_features = np.array([
                [1, 1, 0, 0, 1],
                [1, 1, 0, 0, 0],
                [0, 0, 1, 1, 0],
                [1, 0, 1, 1, 0],
                [0, 0, 0, 0, 1],
                [1, 0, 0, 0, 1]], dtype=np.float32)

users_seen_movies.shape, movies_features.shape

((4, 6), (6, 5))

In [4]:
# 사용자가 본 영화에 대한 평가를 데이터프레임으로 변환
df_users_movie_ratings = pd.DataFrame(users_seen_movies, 
                                      columns=movies, index=users)
df_users_movie_ratings

,스타워즈,다크나이트,쉬렉,서울의 봄,명량,범죄도시
길동,4.0,6.0,8.0,0.0,0.0,0.0
영희,0.0,0.0,10.0,0.0,8.0,3.0
철수,0.0,6.0,0.0,0.0,3.0,7.0
명수,10.0,9.0,0.0,5.0,0.0,2.0


In [5]:
# 영화 전문가가 손수 만든 특성을 데이터프레임으로 변환
df_movies_features = pd.DataFrame(movies_features, 
                                  columns=features, index=movies)
df_movies_features

,액션,공상과학,코미디,만화영화,드라마
스타워즈,1.0,1.0,0.0,0.0,1.0
다크나이트,1.0,1.0,0.0,0.0,0.0
쉬렉,0.0,0.0,1.0,1.0,0.0
서울의 봄,1.0,0.0,1.0,1.0,0.0
명량,0.0,0.0,0.0,0.0,1.0
범죄도시,1.0,0.0,0.0,0.0,1.0


## step1) 사용자 특성 매트릭스 (weighted genre matrix) 계산

사용자의 영화 평가를 근거로 weighted genre matrix 작성 (Genre 별로 평점 합산 및 normalize)

즉, 5 차원 특성 공간에 각 사용자의 임베딩을 포함하는 행렬입니다.  

이것을 `users_movies` matrix와 `movies_features` matrix의 행렬 곱셈으로 계산합니다.

In [6]:
# 사용자가 본 영화의 평가와 영화의 특성을 행렬 곱셈하여 사용자의 특성을 계산
users_features = np.matmul(users_seen_movies, movies_features)

# 사용자 특성 matrix (column - 영화 장르, row - 사용자)
users_features

array([[10., 10.,  8.,  8.,  4.],
       [ 3.,  0., 10., 10., 11.],
       [13.,  6.,  0.,  0., 10.],
       [26., 19.,  5.,  5., 12.]], dtype=float32)

In [7]:
# 계산된 사용자의 특성을 데이터프레임으로 변환
df_users_features = pd.DataFrame(users_features, 
                                 columns=features, index=users)

# 변환된 데이터프레임 출력
df_users_features

,액션,공상과학,코미디,만화영화,드라마
길동,10.0,10.0,8.0,8.0,4.0
영희,3.0,0.0,10.0,10.0,11.0
철수,13.0,6.0,0.0,0.0,10.0
명수,26.0,19.0,5.0,5.0,12.0


## step2) Weighted genre matrix 로부터 User Profile 작성
다음으로 각 사용자 특성 벡터를 정규화하여 1 이 되도록 정규화합니다. 

정규화는 꼭 필요한 것은 아니지만 사용자간에 등급 규모를 비교할 수 있도록 합니다.

In [8]:
# users_features 배열의 각 사용자별로 특징(feature) 값들의 합을 계산합니다.
# axis=1은 열을 따라 연산을 수행하라는 의미이며, 각 사용자별로 모든 특징의 합을 구합니다.
# keepdims=True는 연산 결과의 차원을 원본 배열과 동일하게 유지하라는 옵션입니다.
# 이를 통해, 연산 결과는 원본 배열의 행 형태를 유지하며, 각 행은 해당 사용자의 특징 값들의 합으로 구성됩니다.
np.sum(users_features, axis=1, keepdims=True)

array([[40.],
       [34.],
       [29.],
       [67.]], dtype=float32)

In [9]:
# 사용자 프로파일
# 각 사용자의 특성 벡터를 해당 벡터의 합으로 나누어 정규화합니다.
users_features = users_features / np.sum(users_features, axis=1, keepdims=True)

# 정규화된 사용자의 특성 출력
users_features

array([[0.25      , 0.25      , 0.2       , 0.2       , 0.1       ],
       [0.0882353 , 0.        , 0.29411766, 0.29411766, 0.32352942],
       [0.44827586, 0.20689656, 0.        , 0.        , 0.3448276 ],
       [0.3880597 , 0.2835821 , 0.07462686, 0.07462686, 0.17910448]],
      dtype=float32)

In [10]:
# 사용자 프로파일
# 정규화된 사용자의 특성을 데이터프레임으로 변환
df_users_features = pd.DataFrame(users_features, columns=features, index=users)
df_users_features

,액션,공상과학,코미디,만화영화,드라마
길동,0.250000,0.250000,0.200000,0.200000,0.100000
영희,0.088235,0.000000,0.294118,0.294118,0.323529
철수,0.448276,0.206897,0.000000,0.000000,0.344828
명수,0.388060,0.283582,0.074627,0.074627,0.179104


## step3) 사용자가 보지 않은 Movie matrix 에 User Feature Profile 을 적용하여 추천 영화 list 작성

위에서 계산된 users_feats 를 사용하여 각 사용자에 대한 각 영화 카테고리의 상대적 중요성을 나타낼 수 있습니다.

```
tf.nn.top_k : 마지막 차원에 대한 'k' 가장 큰 항목의 값과 인덱스를 찾습니다.

result = tf.nn.top_k([1, 2, 98, 1, 1, 99, 3, 1, 3, 96, 4, 1], k=3)
>>> result.values.numpy()
array([99, 98, 96], dtype=int32)
>>> result.indices.numpy()
array([5, 2, 9], dtype=int32)
```

각 사용자별로 선호 장르를 순서대로 나열

In [11]:
num_features = len(features)  

# tf.nn.top_k 함수는 입력된 텐서에서 가장 큰 k개의 값을 반환합니다.
# 여기서는 모든 특성을 대상으로 하므로 k에 num_features를 사용합니다.
top_users_features = tf.nn.top_k(users_features, k=num_features)[1]

top_users_features.numpy() 

array([[0, 1, 2, 3, 4],
       [4, 2, 3, 0, 1],
       [0, 4, 1, 2, 3],
       [0, 1, 4, 2, 3]], dtype=int32)

장르 index 를 장르명으로 표시

In [12]:
features

['액션', '공상과학', '코미디', '만화영화', '드라마']

In [13]:
num_users = len(users)  

# 각 사용자에 대해 반복문을 수행합니다.
for i in range(num_users):
    # top_users_features[i]는 i번째 사용자의 특성 인덱스를 담고 있습니다.
    # 해당 인덱스를 사용하여 features 리스트에서 특성 이름을 가져옵니다.
    feature_names = [features[idx] for idx in top_users_features[i]]
    
    # 각 사용자의 이름과 해당 특성 이름을 출력합니다.
    print('{}: {}'.format(users[i], feature_names))

길동: ['액션', '공상과학', '코미디', '만화영화', '드라마']
영희: ['드라마', '코미디', '만화영화', '액션', '공상과학']
철수: ['액션', '드라마', '공상과학', '코미디', '만화영화']
명수: ['액션', '공상과학', '드라마', '코미디', '만화영화']


## 추천할 영화 결정

- 이제 위에서 계산 한 `users_features` 를 사용하여 각 사용자에 대한 영화 등급 및 추천을 결정합니다.  
- 각 영화의 예상 등급을 계산하기 위해 사용자의 특성 벡터와 해당 영화 특성 벡터간의 유사성 측정값을 계산합니다.  

In [14]:
#영화 특성
df_movies_features

,액션,공상과학,코미디,만화영화,드라마
스타워즈,1.0,1.0,0.0,0.0,1.0
다크나이트,1.0,1.0,0.0,0.0,0.0
쉬렉,0.0,0.0,1.0,1.0,0.0
서울의 봄,1.0,0.0,1.0,1.0,0.0
명량,0.0,0.0,0.0,0.0,1.0
범죄도시,1.0,0.0,0.0,0.0,1.0


In [15]:
#사용자 특성
df_users_features

,액션,공상과학,코미디,만화영화,드라마
길동,0.250000,0.250000,0.200000,0.200000,0.100000
영희,0.088235,0.000000,0.294118,0.294118,0.323529
철수,0.448276,0.206897,0.000000,0.000000,0.344828
명수,0.388060,0.283582,0.074627,0.074627,0.179104


### 모든 영화 특성과 모든 사용자 특성간의 유사도 측정  
사용자 특성과 영화 특성 간의 코사인 유사도를 계산하여 예상 평점으 계산합니다.  
코사인 유사도가 높을수록 사용자가 그 영화를 좋아할 가능성이 높기 때문에, 이를 예상 평점으로 간주합니다.  

In [16]:
# 사용자 특성과 영화 특성 간의 코사인 유사도를 예상 평점으로 계산
all_users_ratings = cosine_similarity(users_features, movies_features)
all_users_ratings

array([[0.7470874 , 0.7624929 , 0.6099943 , 0.80934465, 0.21566555,
        0.533745  ],
       [0.4449492 , 0.11677483, 0.7784989 , 0.7309879 , 0.6055301 ,
        0.54494923],
       [0.95871043, 0.7692875 , 0.        , 0.42976674, 0.57259834,
        0.93124276],
       [0.9379619 , 0.9069189 , 0.20153752, 0.592397  , 0.34202054,
        0.7658426 ]], dtype=float32)

In [17]:
# 모든 사용자의 모든 영화에 대한 예상 평점
df_all_users_ratings = pd.DataFrame(all_users_ratings, 
                                    columns=movies, index=users)

df_all_users_ratings

,스타워즈,다크나이트,쉬렉,서울의 봄,명량,범죄도시
길동,0.747087,0.762493,0.609994,0.809345,0.215666,0.533745
영희,0.444949,0.116775,0.778499,0.730988,0.605530,0.544949
철수,0.958710,0.769288,0.000000,0.429767,0.572598,0.931243
명수,0.937962,0.906919,0.201538,0.592397,0.342021,0.765843


- 위의 계산은 데이터베이스의 각 사용자와 각 영화 간의 유사성 측정값을 계산합니다.  
- 새 영화의 등급에만 집중하기 위해 all_users_ratings 매트릭스에 마스크를 적용하여, 사용자가 이미 영화를 평가한 경우 해당 평가를 무시합니다. 이렇게 하면 이전에 본 적이 없거나 등급이 지정되지 않은 영화에 대한 등급에만 집중합니다.

In [18]:
# 사용자가 이미 본 영화에 대한 평가
users_seen_movies

array([[ 4.,  6.,  8.,  0.,  0.,  0.],
       [ 0.,  0., 10.,  0.,  8.,  3.],
       [ 0.,  6.,  0.,  0.,  3.,  7.],
       [10.,  9.,  0.,  5.,  0.,  2.]], dtype=float32)

In [19]:
# 사용자가 보지 않은 영화를 찾기 위해 users_movies 배열과 0으로 이루어진 배열을 비교합니다.
# np.equal 함수는 두 배열의 각 요소를 비교하여 같으면 True, 다르면 False를 반환합니다.
# np.zeros_like(users_movies)는 users_movies와 동일한 형태이며 모든 요소가 0인 배열을 생성합니다.
# 결과적으로, users_unseen_movies 배열은 사용자가 영화를 보지 않았으면 True, 봤으면 False를 가지게 됩니다.
users_unseen_movies = np.equal(users_seen_movies, np.zeros_like(users_seen_movies))
users_unseen_movies

array([[False, False, False,  True,  True,  True],
       [ True,  True, False,  True, False, False],
       [ True, False,  True,  True, False, False],
       [False, False,  True, False,  True, False]])

In [20]:
# np.where 함수를 사용하여 사용자가 보지 않은 영화에 대한 새로운 평점을 할당합니다.
# 조건(users_unseen_movies)이 True인 경우, 즉 사용자가 영화를 보지 않았을 경우에는 all_users_ratings에서 해당하는 값을 사용합니다.
# 조건이 False인 경우, 즉 사용자가 영화를 본 경우에는 zero rating을 할당합니다.
users_ratings_expected = np.where(
    users_unseen_movies,   # 조건: 사용자가 영화를 보지 않았는지 여부 (True: 안 본 영화, False: 본 영화)
    all_users_ratings,     # True일 때 적용할 값: 모든 사용자의 평점(사용자가 보지 않은 영화에 대한 평점)
    0)                     # False일 때 0(이미 본 영화에 대해서는 0 rating 부여)
users_ratings_expected

array([[0.        , 0.        , 0.        , 0.80934465, 0.21566555,
        0.533745  ],
       [0.4449492 , 0.11677483, 0.        , 0.7309879 , 0.        ,
        0.        ],
       [0.95871043, 0.        , 0.        , 0.42976674, 0.        ,
        0.        ],
       [0.        , 0.        , 0.20153752, 0.        , 0.34202054,
        0.        ]], dtype=float32)

In [21]:
# 예상 사용자 평점을 데이터프레임으로 변환
# columns에는 영화 목록을, index에는 사용자 목록을 사용
pd.DataFrame(users_ratings_expected, columns=movies, index=users)

,스타워즈,다크나이트,쉬렉,서울의 봄,명량,범죄도시
길동,0.000000,0.000000,0.000000,0.809345,0.215666,0.533745
영희,0.444949,0.116775,0.000000,0.730988,0.000000,0.000000
철수,0.958710,0.000000,0.000000,0.429767,0.000000,0.000000
명수,0.000000,0.000000,0.201538,0.000000,0.342021,0.000000


- 각 사용자에 대해 보지 않은 영화의 상위 2 개 등급의 영화를 가져와 인쇄해 봅니다.

In [22]:
# 사용자가 예상 평점이 높은 상위 2개의 영화를 선택
# tf.nn.top_k 함수는 주어진 배열에서 가장 높은 값의 인덱스를 반환
top_movies = tf.nn.top_k(users_ratings_expected, 2)[1]

# 상위 2개의 영화 인덱스 출력
top_movies

<tf.Tensor: shape=(4, 2), dtype=int32, numpy=
array([[3, 5],
       [3, 0],
       [0, 3],
       [4, 2]], dtype=int32)>

추천할 영화

In [23]:
# 각 사용자별로 예상 평점이 가장 높은 상위 2개의 영화 제목을 출력
for i in range(num_users):
    # 상위 영화 인덱스를 사용하여 영화 제목을 가져옴
    movie_names = [movies[idx] for idx in top_movies[i]]
    # 사용자 이름과 상위 영화 제목을 출력
    print('{}: {}'.format(users[i], movie_names))

길동: ['서울의 봄', '범죄도시']
영희: ['서울의 봄', '스타워즈']
철수: ['스타워즈', '서울의 봄']
명수: ['명량', '쉬렉']
